In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRe

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-04 03:44:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.57MB/s    in 0.2s    

2023-03-04 03:44:30 (4.57 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdataetl-db").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# **Extract the Amazon Data into Spark DataFrame**

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)

books = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), sep='\t', header=True, inferSchema=True)
books.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [5]:
books.count()

6106719

In [6]:
# Drop null values
books = books.dropna()
print(books.count())

6105983


# **Transform the Data**

In [7]:
# Print schema
books.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

books = books.withColumn("customer_id", books["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", books["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", books["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", books["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", books["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", books["total_votes"].cast(IntegerType()))\

# Print schema
books.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### **Create Review ID Table**

In [9]:
review_id_df = books.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117|2012-05-03 00:00:00|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131|2012-05-03 00:00:00|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025|2012-05-03 00:00:00|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232|2012-05-03 00:00:00|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125|2012-05-03 00:00:00|
| RCLZ5OKZNUSY4|   27780192|146854456X|     270349766|2012-05-03 00:00:00|
|R1S65DJYEI89G4|   13041546|1118094514|     752141158|2012-05-03 00:00:00|
|R3KQYBQOLYDETV|   51692331|0563521147|     729491316|2012-05-03 00:00:00|
|R3QV8K7CSU8K2W|   23108524|0669444421|     261004015|2012-05-03 00:00:00|
|R3W5A1WUGO5VQ0|   51692331|1897784457|     497876045|2012-05-03 00:00:00|
|R20AQCY3FMBVN5|   494382

### **Create Products Table**

In [10]:
products_df = books.select(["product_id", "product_title"]).dropDuplicates(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
|0805087605|What's Inside You...|
|1439197334|Far from Here: A ...|
|1413748260|       Plasma Dreams|
|1613469845|           Bolt Clan|
|B005IUSVS0|       If I Were You|
|0486451550|Architectura Nava...|
|0312426267|Golden Boy: Memor...|
|0425247414|Murder on Fifth A...|
|0375722203|Random House Webs...|
|1615390588|Colodin Project, ...|
|047023847X|HTML, XHTML and C...|
|1892785390|Continuing Care S...|
|1467996149|Far Away in the S...|
|0307886948|Jeannie Out of th...|
|159038668X|The Kingdom and t...|
+----------+--------------------+
only showing top 20 rows



### **Create Customers Table**

In [11]:
from pyspark.sql.functions import desc

customers_df = books.groupby("customer_id").agg({"customer_id" : "count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50122160|         36228|
|   50732546|         16897|
|   43530996|          4405|
|   14410556|          4363|
|   14720400|          3453|
|   44731853|          3430|
|   36642996|          3374|
|   52004451|          2725|
|   45041039|          2508|
|   13161706|          2475|
|   20595117|          2441|
|   52496677|          2263|
|   50776149|          2252|
|   38491967|          2162|
|   52637822|          2159|
|   45273033|          2097|
|   50068216|          2023|
|   34407806|          1858|
|   50881246|          1826|
|   11488357|          1820|
+-----------+--------------+
only showing top 20 rows



### **Create Vine Table**

In [12]:
vine_df = books.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R28HBXXO1UEVJT|          5|            0|          0|   N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|
|R1S65DJYEI89G4|          4|            8|         17|   N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|
|R20AQCY3FMBVN5|          5|            0|          0|   N|
| R7KY8VL871MVL|          5|           13|         15|   N|
| RHF5E4UOL5LQ3|          5|            2|          2|   N|
|R1LMUDN5M9G6ZZ|          5|            

## **Load the DataFrame To RDS**

In [13]:
mode = "append"
jdbc_url="jdbc:postgresql://bigdataetl-db.claqzv8ic3l5.us-east-1.rds.amazonaws.com:5432/big_data_etl_db"
config = {"user":"root", "password": "postgres", "driver":"org.postgresql.Driver"}

In [14]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [15]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [16]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)